In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Particle in a magnetic field #

In class we looked at the motion of a particle in uniform $\vec{B}$-field.  Let us consider this problem numerically.  Given the Lorentz force law it is straightforward to evolve $\vec{x}$ and $\vec{v}$ using a "standard" ODE integrator (e.g. the `solve_ivp` method from `scipy`).  However, this neglects the fact that these equations of motion arise from a Lagrangian or Hamiltonian system, and moreover one that exhibits certain interesting properties.

## Symplectic integrators ##

If the systems of equations we want to evolve come from a Hamiltonian, i.e. they are the equations of motion for some system one can do much, much better than a canned ODE routine using the particular behavior of Hamiltonian flows.

One of the issues with "standard" integrators is that they don't know about the special structure of a Hamiltonian evolution -- the fact that the equations for $\dot{x}$ and $\dot{p}$ are so similar and that very often we have an exactly conserved quantity: $H$.  What you find is that you can get slow drifts of the energy over time, and orbits which should actually close on themselves can start to drift very far from their proper place.

This leads us to consider symplectic integrators.

Suppose we have a single particle moving under a Hamiltonian $H(q,p)=p^2/(2m) + V(q)$ with
$$
  \dot{p} = -\frac{dH}{dq} = -\frac{dV}{dq}
  \quad , \quad
  \dot{q} = \frac{dH}{dp} = \frac{p}{m}
$$
How should we evolve this system?

### Leapfrog ###

The lowest order integrator for such a system is the [leapfrog scheme](https://en.wikipedia.org/wiki/Leapfrog_integration), which interleaves "drifts" of particles with "kicks" due to the forces:

1) $v_{n+1/2} = v_{n} + (h/2) a_n$

2) $q_{n+1} = q_n + h v_{n+1/2}$

3) $v_{n+1} = v_{n+1/2} + (h/2) a_{n+1}$

where $a_n$ is the acceleration at time $n$.  Note that for constant steps you can combine the first (1) and last (3) step into a single "kick" for time $h$.  This algorithm has two advantages: (a) it is explicitly time reversible and (b) it conserves an energy which is close to the physical energy that we want to conserve.

The leapfrog scheme is an example of an "operator splitting" method.

## Particle in a magnetic field ##

Let us be explicit about the problem of interest.  We assume we have a particle,
of charge $q=1$, moving in a potential, $V(\vec{q})$, in the presence of a constant magnetic field.  The Hamiltonian is
$$
  H(\vec{q},\vec{p}) = \frac{\vec{p}^2}{2m} + V(\vec{q})
$$
where we have to be careful in remembering that the __canonical__ momentum $\vec{p}=m\dot{\vec{q}}-(1/2)\mathbf{B}\vec{q}$ since the Lagrangian is
$$
  L(\vec{q},\dot{\vec{q}}) = \frac{1}{2}m\dot{\vec{q}}^2 +
  \frac{1}{2}\vec{q}\mathbf{B}\dot{\vec{q}} - V(\vec{q})
$$
written in terms of the magnetic field matrix
$$
  \mathbf{B} = \left(\begin{array}{ccc}
    0  & -B_z &  B_y \\
   B_z &   0  & -B_x \\
  -B_y &  B_x &   0   \end{array}\right)
$$
The equations of motion are
$$
  \frac{d\vec{q}}{dt} = \frac{\vec{p}}{m} + \frac{1}{2}\mathbf{B}\vec{q}
  \quad , \quad
  \frac{d\vec{p}}{dt} = \frac{1}{2}\mathbf{B}\vec{p}
  - \frac{1}{4m}\mathbf{B}^T\mathbf{B}\vec{q} - \nabla_q V(\vec{q})
$$
For definiteness we'll take $B_x=B_y=0$, so $\vec{B}\propto\hat{z}$.

## Operator splitting ##

We design our symplectic integrator using operator splitting, that is we approximate the evolution as first due to $V(\vec{q})$ and then due to $\vec{p}$ and so on.  Under evolution by $V(\vec{q})$ we have
$$
  \frac{d\vec{q}}{dt} = 0 \quad , \quad
  \frac{d\vec{p}}{dt} = -\nabla_q V(\vec{q})
$$
For a timestep $\Delta t$ starting at step $n$ this means
$$
  \vec{q}' = \vec{q}^n \quad , \quad
  \vec{p}' = \vec{p}^n - \Delta t\ \nabla_q V(\vec{q})
$$
For the kinetic term the equations of motion are
$$
  \frac{d\vec{q}}{dt} = \frac{\vec{p}}{m} \quad , \quad
  \frac{d\vec{p}}{dt} = \frac{\mathbf{B}\vec{p}}{m}
$$
This equation is the same as for our particle without a potential, so we know the solution is a spiral/helix.  We can choose to take a small step.

Putting these together we have
$$
  \vec{p}^{n+1/2} = \vec{p}^n - \frac{1}{2}\Delta t\ \nabla_q V(\vec{q}^n)
$$
followed by
$$
  \vec{q}^{n+1} = \vec{q}^n + \left( \Delta t \mathbf{1}
  + \frac{1}{2}\Delta t^2 \frac{\mathbf{B}}{m} \right) \frac{\vec{p}^{n+1/2}}{m}
$$
and ending with
$$
  \vec{p}^{n+1} = \left(1+\Delta t\frac{\mathbf{B}}{m}+\cdots\right)\vec{p}^{n+1/2}
  -\frac{\Delta t}{2}\nabla_q V(\vec{q}^{n+1})
$$